### Elo
Elo is a method of assigning a numeric value to how good a player is at some game. It is most commonly used in chess. A 400 elo difference means the stronger player should win 9 out of 10 games against the weaker player. 

We construct Elo based on match history. If a player wins a game, their elo will increase, and if they lose, their elo will decrease. This is scaled by the relative strength between the player and their opponent.

Expected Score:

$$E_A = \frac{1}{1 + 10^{(R_B - R_A)/400}}$$

$$E_B = 1 - E_A$$

Actual Score:
- Win: Actual Score = 1
- Lose: Actual Score = 0

The Elo update formula is:

New Rating = Old Rating + K × (Actual Score − Expected Score). 

K controls how fast or slow a player's rating changes after each game. Here we use an adaptive K value.

We also track elo by surface.

### Glicko-2
Glicko-2 is an extension of the Elo rating system. For each player, we track three quantities. 

- Rating (R): Estimated skill
- Rating Deviation (RD): Uncertainty in estimate
- Volatility ($\sigma$): Variation in performance

After every rating period, a player’s rating deviation increases to reflect growing uncertainty. When results come in, the size of the rating update (K factor from elo) is then recalibrated based on multiple variables: your own uncertainty, your opponent’s uncertainty, and both players’ volatility.

This system is derived from treating Elo not like a single value but more like a distribution.

In [2]:
from collections import defaultdict
from typing import Callable, Hashable, Optional

import numpy as np
import pandas as pd
from glicko2 import Player

# read the cleaned dataframe
df = pd.read_parquet("../data/cleaned/atp_matches_cleaned.parquet")
print(df['surface'].unique())

['Hard' 'Carpet' 'Clay' 'Grass']


### Implementation
Below is an implementation of the Glicko-2 algorithm with the glicko2 library.

In [3]:
def add_glicko2_elo(
    df: pd.DataFrame,
    *,
    id_a_col: str = "id_a",
    id_b_col: str = "id_b",
    result_col: str = "result",
    date_col: str = "tourney_date",
    surface_col: str = "surface",
    # default Glicko-2 params for new players
    default_rating: float = 1500.0,
    default_rd: float = 350.0,
    default_vol: float = 0.06,
) -> pd.DataFrame:
    """
    Adds 'elo_a' and 'elo_b' columns with PRE-MATCH Glicko-2 ratings.

    result_col: 1 = A wins, 0 = B wins, 0.5 = draw.
    """

    df = df.sort_values(date_col).reset_index(drop=True).copy()
    df["elo_a"] = np.nan
    df["elo_b"] = np.nan

    df["elo_surface_a"] = np.nan
    df["elo_surface_b"] = np.nan

    # (pool_key, player_id) -> Player()
    players: dict[tuple[Hashable, Hashable], Player] = {}

    def get_player(pool_key: Hashable, pid: Hashable) -> Player:
        key = (pool_key, pid)
        if key not in players:
            players[key] = Player(
                rating=default_rating,
                rd=default_rd,
                vol=default_vol,
            )
        return players[key]

    for idx, row in df.iterrows():
        # global
        p_a = get_player("Global", row[id_a_col])
        p_b = get_player("Global", row[id_b_col])

        # surface specific
        p_a_surface = get_player(row[surface_col], row[id_a_col])
        p_b_surface = get_player(row[surface_col], row[id_b_col])

        # pre-match ratings
        df.at[idx, "elo_a"] = p_a.getRating()
        df.at[idx, "elo_b"] = p_b.getRating()

        df.at[idx, "elo_surface_a"] = p_a_surface.getRating()
        df.at[idx, "elo_surface_b"] = p_b_surface.getRating()

        res = row[result_col]

        # The library's update_player takes lists of opponent ratings/RDs and scores.
        if res == 1:          # A wins
            p_a.update_player([p_b.getRating()], [p_b.getRd()], [1.0])
            p_b.update_player([p_a.getRating()], [p_a.getRd()], [0.0])

            p_a_surface.update_player([p_b_surface.getRating()], [p_b_surface.getRd()], [1.0])
            p_b_surface.update_player([p_a_surface.getRating()], [p_a_surface.getRd()], [0.0])
        elif res == 0:        # B wins
            p_a.update_player([p_b.getRating()], [p_b.getRd()], [0.0])
            p_b.update_player([p_a.getRating()], [p_a.getRd()], [1.0])

            p_a_surface.update_player([p_b_surface.getRating()], [p_b_surface.getRd()], [0.0])
            p_b_surface.update_player([p_a_surface.getRating()], [p_a_surface.getRd()], [1.0])
        else:
            raise ValueError(f"Unsupported result value in '{result_col}': {res}")

    return df

Run the method:

In [4]:
elo_df = add_glicko2_elo(df)
elo_df

,surface,draw_size,tourney_level,tourney_date,id_a,name_a,hand_a,ht_a,age_a,id_b,...,bpFaced_b,rank_a,rank_points_a,rank_b,rank_points_b,result,elo_a,elo_b,elo_surface_a,elo_surface_b
0,Hard,32,A,19910107,101142,Emilio Sanchez,R,180.0,25.6,101746,...,6.0,9.0,1487.0,78.0,459.0,1,1500.000000,1500.000000,1500.000000,1500.000000
1,Hard,32,A,19910107,100923,Wally Masur,R,180.0,27.6,100656,...,6.0,55.0,555.0,3.0,2581.0,1,1500.000000,1500.000000,1500.000000,1500.000000
2,Hard,32,A,19910107,100587,Steve Guy,R,188.0,31.8,101613,...,11.0,220.0,114.0,94.0,371.0,0,1500.000000,1500.000000,1500.000000,1500.000000
3,Hard,32,A,19910107,101601,Brett Steven,R,185.0,21.6,101179,...,8.0,212.0,116.0,77.0,468.0,0,1500.000000,1500.000000,1500.000000,1500.000000
4,Hard,32,A,19910107,101332,Gilad Bloom,L,173.0,23.8,101117,...,12.0,72.0,483.0,65.0,502.0,0,1500.000000,1500.000000,1500.000000,1500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91314,Hard,8,F,20241218,210460,Nishesh Basavareddy,R,180.0,19.6,210506,...,7.0,138.0,440.0,41.0,1245.0,0,1848.067638,1769.185687,1802.242919,1719.578177
91315,Hard,8,F,20241218,209414,Luca Van Assche,R,178.0,20.5,209992,...,13.0,128.0,471.0,50.0,1115.0,1,1575.825877,1742.760750,1538.262162,1685.754055
91316,Hard,8,F,20241218,210150,Jakub Mensik,R,193.0,19.2,210530,...,1.0,48.0,1136.0,122.0,493.0,0,1834.966267,1732.983733,1856.783311,1731.238775
91317,Hard,8,F,20241218,209950,Arthur Fils,R,185.0,20.5,210150,...,7.0,20.0,2355.0,48.0,1136.0,1,1846.168646,1818.386905,1766.910874,1834.331766


### Data Visualisation

We can now do some data visualisation to check if the Elo calculation is accurate.

In [5]:
from datetime import datetime, timedelta
import pandas as pd


def display_top_elo_players_flex(
    df: pd.DataFrame,
    n: int = 10,
    *,
    id_a_col: str = "id_a",
    id_b_col: str = "id_b",
    name_a_col: str = "name_a",
    name_b_col: str = "name_b",
    elo_a_col: str = "elo_a",          # <-- can swap to "elo_surface_a"
    elo_b_col: str = "elo_b",          # <-- can swap to "elo_surface_b"
    date_col: str = "tourney_date",
    surface_col: str = "surface",      # column holding "Grass", "Clay", etc.
    surface: str | None = None,        # e.g. "Grass", "Clay", "Hard"
    exclude_surfaces: tuple = ("Carpet",),
    as_of_date: str | None = None,     # "YYYYMMDD"
    active_window_days: int = 365,
) -> pd.DataFrame:
    """
    Shows top players by a given Elo (overall or surface-specific) as of a given date.
    Only includes players who played within the last `active_window_days`
    before `as_of_date`.

    Parameters
    ----------
    elo_a_col, elo_b_col
        Columns to use for Elo (e.g. "elo_a"/"elo_b" or "elo_surface_a"/"elo_surface_b").
    surface
        If provided, filter to matches on this surface only (e.g. "Grass").
    exclude_surfaces
        Surfaces to drop entirely (defaults to excluding "Carpet").
    """

    df = df.copy()

    # Parse dates
    df[date_col] = df[date_col].astype(str).apply(
        lambda x: datetime.strptime(x, "%Y%m%d")
    )

    # Optional surface filtering
    if surface_col in df.columns:
        if exclude_surfaces:
            df = df[~df[surface_col].isin(exclude_surfaces)]
        if surface is not None:
            df = df[df[surface_col] == surface]

    # Figure out cutoff date
    if as_of_date is None:
        cutoff_date = df[date_col].max()
    else:
        cutoff_date = datetime.strptime(as_of_date, "%Y%m%d")

    active_after = cutoff_date - timedelta(days=active_window_days)

    # Filter to matches up to the as_of_date
    df = df[df[date_col] <= cutoff_date]

    # Build long format using the chosen Elo columns
    a_side = df[[id_a_col, name_a_col, elo_a_col, date_col]].rename(
        columns={
            id_a_col: "player_id",
            name_a_col: "player_name",
            elo_a_col: "elo",
        }
    )
    b_side = df[[id_b_col, name_b_col, elo_b_col, date_col]].rename(
        columns={
            id_b_col: "player_id",
            name_b_col: "player_name",
            elo_b_col: "elo",
        }
    )

    long_df = pd.concat([a_side, b_side], ignore_index=True)
    long_df = long_df.dropna(subset=["elo"])

    # Filter to only players active within X days before cutoff
    recent_matches = long_df[long_df[date_col] >= active_after]
    active_players = set(recent_matches["player_id"].unique())
    long_df = long_df[long_df["player_id"].isin(active_players)]

    # For each player, take their latest Elo before cutoff
    latest_elo = (
        long_df
        .sort_values(date_col)
        .groupby("player_id", as_index=False)
        .last()[["player_id", "player_name", "elo"]]
    )

    top_players = latest_elo.sort_values("elo", ascending=False).head(n)

    print(top_players.to_string(index=False))
    return top_players


We can find the top players by elo, and by surface specific elo.

In [7]:
top10 = display_top_elo_players_flex(elo_df)

 player_id      player_name         elo
    206173    Jannik Sinner 2261.892334
    104925   Novak Djokovic 2127.041669
    207989   Carlos Alcaraz 2089.551836
    100644 Alexander Zverev 2018.926490
    106421  Daniil Medvedev 2013.580313
    104745     Rafael Nadal 1981.664289
    126203     Taylor Fritz 1977.192654
    200282   Alex De Minaur 1976.714394
    207733      Jack Draper 1955.857706
    105777  Grigor Dimitrov 1936.915713


In [8]:
top10 = display_top_elo_players_flex(
    elo_df,
    elo_a_col="elo_surface_a",
    elo_b_col="elo_surface_b",
    surface="Grass",
)

 player_id       player_name         elo
    104925    Novak Djokovic 2089.704523
    207989    Carlos Alcaraz 1951.395616
    126610 Matteo Berrettini 1904.327605
    206173     Jannik Sinner 1848.025007
    104918       Andy Murray 1829.672514
    105683      Milos Raonic 1806.063050
    106421   Daniil Medvedev 1773.559876
    207733       Jack Draper 1772.750887
    126094     Andrey Rublev 1770.206450
    128034    Hubert Hurkacz 1766.502478
